In [1]:
import numpy as np
import cv2
from matplotlib import pyplot as plt
import numpy as np
from PIL import Image
import random
#import dlib
# import stasm

%matplotlib inline

%load_ext autoreload
%autoreload 2

In [2]:
PREDICTOR_PATH = "./shape_predictor_68_face_landmarks.dat"

detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor(PREDICTOR_PATH)

def get_landmarks(im):
    rects = detector(im, 1)
    
    if len(rects) > 1:
        raise TooManyFaces
    if len(rects) == 0:
        raise NoFaces

    return np.array([[p.x, p.y] for p in predictor(im, rects[0]).parts()])

def read_crop_image(image_path):
    # image_pil = Image.open(image_path) #.convert('L')
    image_pil = cv2.imread(image_path)
    # Convert the image format into numpy array
    # image = np.array(image_pil, 'uint8')

    gray = cv2.cvtColor(image_pil, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, 1.3, 5)
    for (x,y,w,h) in faces:
        #cv2.rectangle(img,(x,y),(x+w,y+h),(255,0,0),2)
        roi_gray = gray[y:y+h, x:x+w]

    image = np.array(roi_gray, 'uint8')
    return image

NameError: name 'dlib' is not defined

In [5]:
lanmark_pts = get_landmarks(img)
lanmark_pts = lanmark_pts.flatten()
lanmark_pts

array([ 62, 131,  64, 148,  66, 164,  70, 180,  76, 195,  86, 208,  97,
       220, 109, 228, 125, 230, 141, 229, 155, 221, 167, 210, 177, 198,
       183, 183, 187, 167, 190, 150, 192, 132,  72, 119,  80, 110,  93,
       107, 107, 109, 120, 114, 136, 113, 148, 108, 161, 106, 174, 109,
       181, 119, 128, 128, 128, 141, 128, 153, 128, 166, 114, 172, 120,
       174, 127, 177, 134, 175, 141, 173,  86, 129,  93, 126, 102, 126,
       110, 131, 102, 132,  93, 132, 145, 130, 153, 125, 162, 125, 169,
       129, 162, 132, 153, 132, 102, 191, 110, 187, 119, 186, 125, 188,
       132, 187, 140, 190, 147, 196, 139, 202, 131, 205, 124, 204, 117,
       203, 110, 198, 106, 191, 119, 191, 125, 192, 132, 192, 143, 195,
       131, 196, 125, 196, 118, 194])

In [6]:
import os
import cv2
import numpy as np
from PIL import Image
path = "/Users/mohamed/Downloads/jaffe"
images = []
labels = []

face_cascade = cv2.CascadeClassifier('./params/haarcascade_frontalface_default.xml')
left_eye_cascade = cv2.CascadeClassifier('./params/haarcascade_lefteye_2splits.xml')
right_eye_cascade = cv2.CascadeClassifier('./params/haarcascade_righteye_2splits.xml')
sift = cv2.SIFT()

# labels = {'Neutral': 0, 'Happy': 1, 'Sad': 2, 'Surprise': 3, 'Angry': 4, 'Disgust': 5, 'Fear': 6}
labelsDic = {'NE': 0, 'HA': 1, 'SA': 2, 'SU': 3, 'AN': 4, 'DI': 5, 'FE': 6, 'CO': 7}


# neutral image .. S506.NE.002.00000001.png
neutral_image = read_crop_image("/Users/mohamed/Documents/999-images/01.HA.123.jpg")

# """"""
def load_jaffe_dataset(path):
    images_paths = [os.path.join(path, f) for f in os.listdir(path) if  f.endswith('.png') or f.endswith('.tiffddd')]
    scale_factor = 4
    for image_path in images_paths:
        
        # image_pil = Image.open(image_path) #.convert('L')
        image_pil = cv2.imread(image_path)
        # Convert the image format into numpy array
        # image = np.array(image_pil, 'uint8')

        gray = cv2.cvtColor(image_pil, cv2.COLOR_BGR2GRAY)
        faces = face_cascade.detectMultiScale(gray, 1.3, 5)
        for (x,y,w,h) in faces:
            #cv2.rectangle(img,(x,y),(x+w,y+h),(255,0,0),2)
            roi_gray = gray[y:y+h, x:x+w]

        image = np.array(roi_gray, 'uint8')
        
        image_label_nbr = int(labelsDic.get(os.path.split(image_path)[1].split(".")[1][:2]))
        landmark_vector = get_landmarks(image)
        # add X-Y  diff of each image and neutreal image a feature too
        landmark_vector_neutral = get_landmarks(neutral_image)
        X = np.array([x[0] for x in landmark_vector])
        Y = np.array([x[1] for x in landmark_vector])
        
        XNE = np.array([x[0] for x in landmark_vector_neutral])
        YNE = np.array([x[1] for x in landmark_vector_neutral])
        
        XXNE = np.subtract(X,XNE)
        YYNE = np.subtract(Y,YNE)
        
        XXNE = np.expand_dims(XXNE, axis=0)
        YYNE = np.expand_dims(YYNE, axis=0)
        
        landmark_vector_augm = np.concatenate((landmark_vector, XXNE.T), axis=1)
        landmark_vector_augm = np.concatenate((landmark_vector_augm, YYNE.T), axis=1)
        images.append(landmark_vector.flatten())
        labels.append(image_label_nbr)
    return images, labels

In [7]:

images, labels = load_jaffe_dataset(path)

In [12]:
images = np.array(images, 'float64') #np.reshape(images, (images[]))
images.shape

(328, 272)

In [13]:
# Third-party libraries
from sklearn.grid_search import GridSearchCV
from sklearn import svm
from sklearn.cross_validation import train_test_split
from sklearn.svm import SVC
from time import time

def svm_baseline():
    # split into a training and testing set
    X_train, X_test, y_train, y_test = train_test_split(
    images, labels, test_size=0.25, random_state=42)
    print  X_train.shape
    print  X_test.shape
    print  len(y_train)
    print  len(y_test)

    # train
    t0 = time()
    param_grid = {'C': [1,4,8, 8.5, 1e3, 5e3, 1e4, 5e4, 1e5],
                  'gamma': [0.0001, 0.0005, 0.001, 0.005, 0.01, 0.1], }
    clf = GridSearchCV(SVC(kernel='rbf', class_weight='balanced'), param_grid)
    clf = clf.fit(X_train, y_train)
    print("done in %0.3fs" % (time() - t0))
    print("Best estimator found by grid search:")
    print(clf.best_estimator_)

    #clf = svm.SVC(C=8.5, gamma=0.0001)
    #clf.fit(X_train, y_train)
    # test
    predictions = [int(a) for a in clf.predict(X_test)]
    num_correct = sum(int(a == y) for a, y in zip(predictions, y_test))
    print "Baseline classifier using an SVM."
    print "%s of %s values correct." % (num_correct, len(y_test))
    print "accuracy is %f (%%)" % (100 * num_correct * 1.0 / len(y_test))

if __name__ == "__main__":
    svm_baseline()

(246, 272)
(82, 272)
246
82
done in 3.827s
Best estimator found by grid search:
SVC(C=4, cache_size=200, class_weight='balanced', coef0=0.0,
  decision_function_shape=None, degree=3, gamma=0.0001, kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)
Baseline classifier using an SVM.
55 of 82 values correct.
accuracy is 67.073171 (%)


In [18]:
# Third-party libraries
from sklearn.grid_search import GridSearchCV
from sklearn import svm
from sklearn.cross_validation import train_test_split
from sklearn.svm import SVC
from time import time

def svm_baseline():
    # split into a training and testing set
    X_train, X_test, y_train, y_test = train_test_split(
    images, labels, test_size=0.25, random_state=42)
    print  X_train.shape
    print  X_test.shape
    print  len(y_train)
    print  len(y_test)

    # train
    t0 = time()
    clf = svm.SVC(C=80.5, gamma=0.0001)
    clf.fit(X_train, y_train)
    print("done in %0.3fs" % (time() - t0))
    # test
    predictions = [int(a) for a in clf.predict(X_test)]
    num_correct = sum(int(a == y) for a, y in zip(predictions, y_test))
    print "Baseline classifier using an SVM."
    print "%s of %s values correct." % (num_correct, len(y_test))
    print "accuracy is %f (%%)" % (100 * num_correct * 1.0 / len(y_test))

if __name__ == "__main__":
    svm_baseline()

(246, 272)
(82, 272)
246
82
done in 0.042s
Baseline classifier using an SVM.
55 of 82 values correct.
accuracy is 67.073171 (%)


In [177]:
X_train, X_test, y_train, y_test = train_test_split(
    images, labels, test_size=0.25, random_state=42)
mean_image = np.mean(X_train, axis=0)
X_train -= mean_image
X_test -= mean_image
X_train = np.hstack([X_train, np.ones((X_train.shape[0], 1))])
X_test = np.hstack([X_test, np.ones((X_test.shape[0], 1))])

X_train


array([[ -4.03950617,  13.69382716,  -4.03950617, ...,  31.8       ,
         36.43703704,   1.        ],
       [ -8.03950617, -27.30617284,  -8.03950617, ..., -25.2       ,
        -58.56296296,   1.        ],
       [  2.96049383,   0.69382716,   2.96049383, ...,  20.8       ,
         18.43703704,   1.        ],
       ..., 
       [ 20.96049383,  15.69382716,  20.96049383, ...,  25.8       ,
         72.43703704,   1.        ],
       [ -6.03950617,  16.69382716,  -6.03950617, ...,  17.8       ,
         46.43703704,   1.        ],
       [-10.03950617, -36.30617284, -10.03950617, ..., -38.2       ,
        -61.56296296,   1.        ]])

In [179]:
def svm_baseline2(X_train, X_test, y_train, y_test):
    
    print  X_train.shape
    print  X_test.shape
    print  len(y_train)
    print  len(y_test)

    # train
    clf = svm.SVC(C=8.5, gamma=0.0001)
    clf.fit(X_train, y_train)
    # test
    predictions = [int(a) for a in clf.predict(X_test)]
    num_correct = sum(int(a == y) for a, y in zip(predictions, y_test))
    for a, y in zip(predictions, y_test):
        print "prediction %d, lebel %d" % (a, y)
        
    print "Baseline classifier using an SVM."
    print "%s of %s values correct." % (num_correct, len(y_test))

if __name__ == "__main__":
    svm_baseline()

(405, 272)
(136, 272)
405
136
Baseline classifier using an SVM.
83 of 136 values correct.
accuracy is 61.029412 (%)


In [3]:
import os
path = "/Users/mohamed/Downloads/jaffe"
images_paths = [os.path.join(path, f) for f in os.listdir(path) if f.endswith('.tiff') or f.endswith('.png') ]
len(images_paths)

540